In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time # Dùng để sleep chương trình
import pandas as pd # Dùng để đọc và hiển thị file csv (Pandas sẽ được học chi tiết ở buổi tới)
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re

In [35]:
#Test new version by Long
url = "https://tuoitre.vn/timeline/0/trang-1.htm"
html_text = requests.get(url).text
#Code below is just copy from above
soup = BeautifulSoup(html_text, 'html.parser')
titles = soup.findAll('h3', class_='title-news')
links = [link.find('a').attrs["href"] for link in titles]
category=soup.findAll('a', class_='category-name fl mgl10 mgb4 uppercase')
category_column=[]
for x in category:
    category_column+=[x.text]

In [36]:
#Test new version by Long
url = "https://tuoitre.vn/timeline/0/trang-1.htm"
html_text = requests.get(url).text
#Code below is just copy from above
soup = BeautifulSoup(html_text, 'html.parser')
news_item=soup.findAll('li', class_='news-item')
titles=[]
links=[]
category_column=[]
for item in news_item:
    titles+=[item.find('h3', class_='title-news').text.replace('\n','')]
    links+=[item.find('a').attrs["href"]]
    category_column+=[item.find('a', class_='category-name fl mgl10 mgb4 uppercase').text]

category_column[0]


'Giải trí'

In [6]:
def links_scraping(iter_num, num, sleep_time = 1):
    continue_flag = True
    links = []
    category_column=[]
    for index in range(iter_num*num+1,(iter_num+1)*num+1):
        url = f"https://tuoitre.vn/timeline/0/trang-{index}.htm"
        html_text = requests.get(url).text
        
        soup = BeautifulSoup(html_text, 'html.parser')
        news_item=soup.findAll('li', class_='news-item')
        if (len(news_item) == 0):
            continue_flag = False
            print('Loi o trang ',index)
            print(html_text)
            break
        for item in news_item:
            ###############################################################
            links+=[item.find('a').attrs["href"]]
            try:
                category=item.find('a', class_='category-name').text
            except:
                category=''
            category_column+=[category]
        #Sleep
        if (index%10==0): print(f"Page {index} complete!")
        time.sleep(sleep_time)
    return (links,category_column,continue_flag)

def content_scraping(links, sleep_time = 1):
    title_column=[]
    description_column=[]
    content_column=[]
    for link in links:
        news = requests.get("https://tuoitre.vn" + link)
        soup= BeautifulSoup(news.content, "html.parser")
        try:
            title=soup.find("h1", class_="article-title").text
        except:
            title=''
        title_column+=[title]
        try:
            description=soup.find("h2", class_="sapo").text
        except:
            description=''
        description_column+=[description]
        body = soup.find("div", id="main-detail-body")
        try:
            content=body.findChildren("p", recursive=False)
            contents=''
            for x in content:
                contents+=x.text
        except:
            contents=''
        content_column+=[contents]
        time.sleep(sleep_time)
    return (title_column,description_column,content_column)

def export_scraped(links,title_column,description_column,content_column,category_column,batch_num):
    #Convert scraped data -> data frame
    df=pd.DataFrame()
    df['links']=links
    df['title']=title_column
    df['description']=description_column
    df['content']=content_column
    df['class']=category_column
    #Export for future use
    df.to_csv(f'scraped_data\crawling_{batch_num}.csv',index=False,encoding="utf-8")

In [7]:
#Batch scraping
iter_num = 12
continue_flag = True

while (continue_flag):
    links,category_column,continue_flag = links_scraping(iter_num, 200,0)
    title_column,description_column,content_column = content_scraping(links,0)
    export_scraped(links,title_column,description_column,content_column,category_column,iter_num)
    if (not continue_flag): break
    else: iter_num+=1

Page 2410 complete!
Page 2420 complete!
Page 2430 complete!
Page 2440 complete!
Page 2450 complete!
Page 2460 complete!
Page 2470 complete!
Page 2480 complete!
Page 2490 complete!
Page 2500 complete!
Page 2510 complete!
Page 2520 complete!
Page 2530 complete!
Page 2540 complete!
Page 2550 complete!
Page 2560 complete!
Page 2570 complete!
Page 2580 complete!
Page 2590 complete!
Page 2600 complete!


KeyboardInterrupt: 

In [5]:
title_column=[]
description_column=[]
content_column=[]
for link in links:
    news = requests.get("https://tuoitre.vn" + link)
    soup= BeautifulSoup(news.content, "html.parser")
    try:
        title=soup.find("h1", class_="article-title").text
    except:
        title=''
    title_column+=[title]
    try:
        description=soup.find("h2", class_="sapo").text
    except:
        description=''
    description_column+=[description]
    body = soup.find("div", id="main-detail-body")
    try:
        content=body.findChildren("p", recursive=False)
        contents=''
        for x in content:
            contents+=x.text
    except:
        contents=''
    content_column+=[contents]

In [8]:
#Convert scraped data -> data frame
df=pd.DataFrame()
df['links']=links
df['title']=title_column
df['description']=description_column
df['content']=content_column
df['class']=category_column
df.head(5)

,links,title,description,content,class
0,/nhu-chua-he-co-cuoc-chia-ly-gap-mat-con-mot-l...,Như chưa hề có cuộc chia ly: ‘Gặp mặt con một ...,TTO - Sau 51 năm đi khắp nơi tìm kiếm con trai...,Đây là một trong số những trường hợp trẻ em th...,Giải trí
1,/hai-nu-sinh-bi-danh-hoi-dong-tren-pho-phong-g...,"Hai nữ sinh bị đánh hội đồng trên phố, Phòng G...","TTO - Ngày 4-1, Phòng Giáo dục và đào tạo quận...",Theo thông tin xác nhận từ Phòng Giáo dục và đ...,Giáo dục
2,/diem-tin-the-thao-toi-4-1-cuu-gdkt-bong-da-nh...,Điểm tin thể thao tối 4-1: Cựu GĐKT bóng đa...,"TTO - CLB Sài Gòn có cố vấn cấp cao ""xịn"", CLB...",Cựu GĐKT bóng đá Nhật Bản làm cố vâ...,Thể thao
3,/iran-bat-tau-cho-hoa-chat-han-quoc-co-thuyen-...,Iran bắt tàu chở hóa chất Hàn Quốc có thuyền v...,TTO - Truyền thông Iran ngày 4-1 cho biết Vệ b...,Một số hãng truyền thông Iran cho biết IRGC đã...,Thế giới
4,/canh-bao-chieu-muon-danh-ngan-hang-moi-du-hoi...,Cảnh báo chiêu mượn danh ngân hàng mời dự hội ...,TTO - Giả danh ngân hàng gọi điện thoại mời mở...,"VPBank vừa phát đi cảnh báo trên vào hôm nay, ...",Kinh doanh


In [49]:
#Export for future use
df.to_csv('crawling.csv',index=False,encoding="utf-8")